In [10]:
# importing the zipfile module 
from zipfile import ZipFile 
  
# loading the temp.zip and creating a zip object 
with ZipFile('train_split.zip', 'r') as zObject: 
  
    # Extracting all the members of the zip  
    # into a specific location. 
    zObject.extractall( 
        path="./") 

In [3]:
!pip install --upgrade --force-reinstall "numpy<2"
!pip install --upgrade pybind11>=2.12
# Install required packages (omit tf-keras to prevent TF integration)
!pip install transformers datasets accelerate peft jiwer soundfile

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 90.8 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: numpy
    Found existing installation: numpy 2.1.3
    Uninstalling numpy-2.1.3:
      Successfully uninstalled numpy-2.1.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scipy 1.9.3 requires numpy<1.26.0,>=1.18.5, but you have numpy 1.26.4 which is incompatible.

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A ne

In [1]:
import os
import json
import numpy as np
import soundfile as sf
from datasets import load_dataset, Audio
from transformers import WhisperProcessor, WhisperForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer
from peft import LoraConfig, get_peft_model, TaskType
import jiwer

2025-07-03 12:59:31.598001: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751547571.611221     475 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751547571.615142     475 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1751547571.626687     475 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1751547571.626707     475 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1751547571.626709     475 computation_placer.cc:177] computation placer alr

In [13]:
pip install numpy==1.24.3

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 91.0 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.19.0 requires numpy<2.2.0,>=1.26.0, but you have numpy 1.24.3 which is incompatible.

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [8]:
!pip install librosa soundfile

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.7/260.7 kB 37.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 430.0/430.0 kB 76.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 101.9 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.6/64.6 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.5/252.5 kB 70.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 MB 72.8 MB/s eta 0:00:00:00:0100:01

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [22]:
#DATA_ROOT = 'data'
OUT_DIR = 'hf_manifests'
os.makedirs(OUT_DIR, exist_ok=True)

for split in ['train_split', 'test_split']:
    transcripts_dir = os.path.join(split, 'transcripts')
    manifest_path = os.path.join(OUT_DIR, f'{split}.jsonl')
    with open(manifest_path, 'w') as outf:
        # load mapping
        wav_scp = os.path.join(transcripts_dir, 'wav.scp')
        segments_f = os.path.join(transcripts_dir, 'segments')
        text_f = os.path.join(transcripts_dir, 'text')
        # build dicts
        wav_map = dict(line.strip().split(' ', 1) for line in open(wav_scp))
        segments = {u: (r, float(s), float(e)) for u, r, s, e in (l.split() for l in open(segments_f))}
        transcripts = {}
        for line in open(text_f):
            parts = line.strip().split(' ', 1)
            transcripts[parts[0]] = parts[1] if len(parts) > 1 else ''
        # write manifest entries
        for utt, (rec, st, ed) in segments.items():
            entry = {
                'id': utt,
                'audio': {
                    'path': os.path.join(transcripts_dir, wav_map[rec]),
                    'offset': st,
                    'duration': ed - st
                },
                'transcript': transcripts.get(utt, '')
            }
            outf.write(json.dumps(entry) + '\n')
    print(f"Written manifest: {manifest_path}")

Written manifest: hf_manifests/train_split.jsonl
Written manifest: hf_manifests/test_split.jsonl


# Load dataset + init Whisper-small + LoRA

In [24]:
data_files = {split: os.path.join(OUT_DIR, f"{split}.jsonl") for split in ['train_split', 'test_split']}
ds = load_dataset('json', data_files=data_files, split=None)
# Cast audio column to HF Audio feature
for split in ds:
    ds[split] = ds[split].cast_column('audio', Audio(sampling_rate=16000))

# Initialize Whisper model & processor
model_name = 'openai/whisper-small'
processor = WhisperProcessor.from_pretrained(model_name, language='hindi', task='transcribe')
model = WhisperForConditionalGeneration.from_pretrained(model_name)
# Freeze base model params
for param in model.parameters():
    param.requires_grad = False
# Set decoder prompt IDs
model.config.forced_decoder_ids = processor.get_decoder_prompt_ids(language='hindi', task='transcribe')

# Configure and apply LoRA
lora_config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,
    inference_mode=False,
    r=16,
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=['q_proj', 'v_proj']
)
model = get_peft_model(model, lora_config)
model

Generating train_split split: 0 examples [00:00, ? examples/s]

Generating test_split split: 0 examples [00:00, ? examples/s]

PeftModelForSeq2SeqLM(
  (base_model): LoraModel(
    (model): WhisperForConditionalGeneration(
      (model): WhisperModel(
        (encoder): WhisperEncoder(
          (conv1): Conv1d(80, 768, kernel_size=(3,), stride=(1,), padding=(1,))
          (conv2): Conv1d(768, 768, kernel_size=(3,), stride=(2,), padding=(1,))
          (embed_positions): Embedding(1500, 768)
          (layers): ModuleList(
            (0-11): 12 x WhisperEncoderLayer(
              (self_attn): WhisperSdpaAttention(
                (k_proj): Linear(in_features=768, out_features=768, bias=False)
                (v_proj): lora.Linear(
                  (base_layer): Linear(in_features=768, out_features=768, bias=True)
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.1, inplace=False)
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=768, out_features=16, bias=False)
                  )
                  (lora_B):

# Preprocess → features & labels

Does: preprocess() extracts log-Mel features via processor.feature_extractor and tokenizes transcripts; maps to train_ds & test_ds

In [25]:
def preprocess(batch):
    audio = batch['audio']
    # extract input features
    feats = processor.feature_extractor(
        audio['array'], sampling_rate=audio['sampling_rate'], return_tensors='pt'
    ).input_features[0]
    # tokenize transcripts
    labels = processor.tokenizer(
        batch['transcript'], return_tensors='pt', padding=False
    ).input_ids[0]
    return {'input_features': feats, 'labels': labels}

# Map preprocessing
train_ds = ds['train_split'].map(preprocess, remove_columns=ds['train_split'].column_names)
test_ds  = ds['test_split'].map(preprocess,  remove_columns=ds['test_split'].column_names)

Map:   0%|          | 0/209 [00:00<?, ? examples/s]

Map:   0%|          | 0/209 [00:00<?, ? examples/s]

# Collate v1 (pad features & labels)

Does: Custom collate_fn that pads input_features to same time length and pads labels to same length with -100 (CTC/seq2seq loss ignore index)

In [29]:
from torch.optim import AdamW
from torch.utils.data import DataLoader
from transformers import get_linear_schedule_with_warmup

def collate_fn(batch):
    import torch
    # Convert lists to tensors and collect feats
    feats = []
    for b in batch:
        f = b['input_features']
        # convert to tensor
        if not torch.is_tensor(f):
            f = torch.tensor(f, dtype=torch.float)
        feats.append(f)
    # Pad feats
    max_t = max(f.shape[0] for f in feats)
    feat_dim = feats[0].shape[1]
    padded_feats = []
    for f in feats:
        pad_len = max_t - f.shape[0]
        if pad_len > 0:
            pad_tensor = torch.zeros((pad_len, feat_dim), dtype=f.dtype)
            f = torch.cat([f, pad_tensor], dim=0)
        padded_feats.append(f)
    input_features = torch.stack(padded_feats)
    # Process labels
    labels = []
    for b in batch:
        l = b['labels']
        if not torch.is_tensor(l):
            l = torch.tensor(l, dtype=torch.long)
        labels.append(l)
    max_l = max(l.shape[0] for l in labels)
    padded_labels = []
    for l in labels:
        pad_len = max_l - l.shape[0]
        if pad_len > 0:
            pad_tensor = torch.full((pad_len,), -100, dtype=l.dtype)
            l = torch.cat([l, pad_tensor], dim=0)
        padded_labels.append(l)
    labels = torch.stack(padded_labels)
    return {'input_features': input_features, 'labels': labels}

# Train v1 (no attention_mask) + eval

Does: Builds DataLoaders; AdamW + linear warmup; unwraps underlying Whisper module; trains 5 epochs; saves outputs/whisper_lora_trained.pth; generates on test; computes WER (used as MER proxy) with jiwer

In [32]:
from torch.optim import AdamW
from torch.utils.data import DataLoader
from transformers import get_linear_schedule_with_warmup

def collate_fn(batch):
    import torch
    feats = []
    for b in batch:
        f = b['input_features']
        if not torch.is_tensor(f): f = torch.tensor(f, dtype=torch.float)
        feats.append(f)
    max_t = max(f.shape[0] for f in feats)
    feat_dim = feats[0].shape[1]
    padded_feats = []
    for f in feats:
        pad_len = max_t - f.shape[0]
        if pad_len > 0:
            pad_tensor = torch.zeros((pad_len, feat_dim), dtype=f.dtype)
            f = torch.cat([f, pad_tensor], dim=0)
        padded_feats.append(f)
    input_features = torch.stack(padded_feats)
    labels = []
    for b in batch:
        l = b['labels']
        if not torch.is_tensor(l): l = torch.tensor(l, dtype=torch.long)
        labels.append(l)
    max_l = max(l.shape[0] for l in labels)
    padded_labels = []
    for l in labels:
        pad_len = max_l - l.shape[0]
        if pad_len > 0:
            pad_tensor = torch.full((pad_len,), -100, dtype=l.dtype)
            l = torch.cat([l, pad_tensor], dim=0)
        padded_labels.append(l)
    labels = torch.stack(padded_labels)
    return {'input_features': input_features, 'labels': labels}

# Prepare DataLoader, optimizer, scheduler
device = 'cuda' if torch.cuda.is_available() else 'cpu'
batch_size = 8
dl_train = DataLoader(train_ds, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
optimizer = AdamW(model.parameters(), lr=1e-4)
sched = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=500, num_training_steps=len(dl_train)*5)

# Determine underlying Whisper model to avoid wrapper issues
if hasattr(model, 'model'):
    whisper_model = model.model
elif hasattr(model, 'base_model'):
    whisper_model = model.base_model
else:
    whisper_model = model
whisper_model.to(device)
whisper_model.train()

epochs = 5
for epoch in range(epochs:=epochs if 'epochs' in locals() else 5):
    total_loss = 0.0
    for batch in dl_train:
        optimizer.zero_grad()
        feats = batch['input_features'].to(device)
        labs = batch['labels'].to(device)
        out = whisper_model(input_features=feats, labels=labs)
        loss = out.loss
        loss.backward()
        optimizer.step()
        sched.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs} avg loss: {total_loss/len(dl_train):.4f}")

# Save model state
torch.save(model.state_dict(), 'outputs/whisper_lora_trained.pth')

# Evaluation on test set
dl_test = DataLoader(test_ds, batch_size=4, collate_fn=collate_fn)
whisper_model.eval()
preds, refs = [], []
with torch.no_grad():
    for batch in dl_test:
        feats = batch['input_features'].to(device)
        gen_ids = whisper_model.generate(input_features=feats)
        preds.extend(processor.batch_decode(gen_ids, skip_special_tokens=True))
        refs.extend(processor.batch_decode(batch['labels'], skip_special_tokens=True))
# Compute WER as MER proxy
wer_score = jiwer.wer(refs, preds)
print(f"Test WER/MER: {wer_score:.2%}")

Epoch 1/5 avg loss: 3.2596
Epoch 2/5 avg loss: 3.2820
Epoch 3/5 avg loss: 3.0617
Epoch 4/5 avg loss: 3.0248
Epoch 5/5 avg loss: 2.9147


Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Test WER/MER: 509.50%


In [33]:
## Changes attention_mask

# Collate v2 (adds attention_mask)

Does: Rewrites collate_fn to also create a boolean attention_mask aligned with padded input_features; pads labels with -100

In [34]:
from torch.optim import AdamW
from torch.utils.data import DataLoader
from transformers import get_linear_schedule_with_warmup

def collate_fn(batch):
    import torch
    # Convert and collect input_features
    feats = []
    masks = []
    for b in batch:
        f = b['input_features']
        # to tensor
        if not torch.is_tensor(f): f = torch.tensor(f, dtype=torch.float)
        feats.append(f)
    # pad feats and build attention_mask
    max_t = max(f.shape[0] for f in feats)
    feat_dim = feats[0].shape[1]
    padded_feats = []
    for f in feats:
        seq_len = f.shape[0]
        pad_len = max_t - seq_len
        # pad frames
        if pad_len > 0:
            pad = torch.zeros((pad_len, feat_dim), dtype=f.dtype)
            pf = torch.cat([f, pad], dim=0)
        else:
            pf = f
        padded_feats.append(pf)
        # mask: 1 for real, 0 for pad
        mask = torch.cat([torch.ones(seq_len, dtype=torch.bool), torch.zeros(max_t-seq_len, dtype=torch.bool)])
        masks.append(mask)
    input_features = torch.stack(padded_feats)
    attention_mask = torch.stack(masks)
    # Process labels
    labels = []
    for b in batch:
        l = b['labels']
        if not torch.is_tensor(l): l = torch.tensor(l, dtype=torch.long)
        labels.append(l)
    max_l = max(l.shape[0] for l in labels)
    padded_labels = []
    for l in labels:
        pad_len = max_l - l.shape[0]
        if pad_len > 0:
            pad = torch.full((pad_len,), -100, dtype=l.dtype)
            pl = torch.cat([l, pad], dim=0)
        else:
            pl = l
        padded_labels.append(pl)
    labels = torch.stack(padded_labels)
    return {'input_features': input_features, 'attention_mask': attention_mask, 'labels': labels}


# Train v2 (with attention_mask) + eval

Does: Same loop as Cell 9 but (intended to) pass attention_mask (note: this cell’s loop still only shows input_features in generate()); evaluates WER/MER

In [35]:
from torch.optim import AdamW
from torch.utils.data import DataLoader
from transformers import get_linear_schedule_with_warmup

def collate_fn(batch):
    import torch
    feats = []
    for b in batch:
        f = b['input_features']
        if not torch.is_tensor(f): f = torch.tensor(f, dtype=torch.float)
        feats.append(f)
    max_t = max(f.shape[0] for f in feats)
    feat_dim = feats[0].shape[1]
    padded_feats = []
    for f in feats:
        pad_len = max_t - f.shape[0]
        if pad_len > 0:
            pad_tensor = torch.zeros((pad_len, feat_dim), dtype=f.dtype)
            f = torch.cat([f, pad_tensor], dim=0)
        padded_feats.append(f)
    input_features = torch.stack(padded_feats)
    labels = []
    for b in batch:
        l = b['labels']
        if not torch.is_tensor(l): l = torch.tensor(l, dtype=torch.long)
        labels.append(l)
    max_l = max(l.shape[0] for l in labels)
    padded_labels = []
    for l in labels:
        pad_len = max_l - l.shape[0]
        if pad_len > 0:
            pad_tensor = torch.full((pad_len,), -100, dtype=l.dtype)
            l = torch.cat([l, pad_tensor], dim=0)
        padded_labels.append(l)
    labels = torch.stack(padded_labels)
    return {'input_features': input_features, 'labels': labels}

# Prepare DataLoader, optimizer, scheduler
device = 'cuda' if torch.cuda.is_available() else 'cpu'
batch_size = 8
dl_train = DataLoader(train_ds, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
optimizer = AdamW(model.parameters(), lr=1e-4)
sched = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=500, num_training_steps=len(dl_train)*5)

# Determine underlying Whisper model to avoid wrapper issues
if hasattr(model, 'model'):
    whisper_model = model.model
elif hasattr(model, 'base_model'):
    whisper_model = model.base_model
else:
    whisper_model = model
whisper_model.to(device)
whisper_model.train()

epochs = 5
for epoch in range(epochs:=epochs if 'epochs' in locals() else 5):
    total_loss = 0.0
    for batch in dl_train:
        optimizer.zero_grad()
        feats = batch['input_features'].to(device)
        labs = batch['labels'].to(device)
        out = whisper_model(input_features=feats, labels=labs)
        loss = out.loss
        loss.backward()
        optimizer.step()
        sched.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs} avg loss: {total_loss/len(dl_train):.4f}")

# Save model state
torch.save(model.state_dict(), 'outputs/whisper_lora_trained.pth')

# Evaluation on test set
dl_test = DataLoader(test_ds, batch_size=4, collate_fn=collate_fn)
whisper_model.eval()
preds, refs = [], []
with torch.no_grad():
    for batch in dl_test:
        feats = batch['input_features'].to(device)
        gen_ids = whisper_model.generate(input_features=feats)
        preds.extend(processor.batch_decode(gen_ids, skip_special_tokens=True))
        refs.extend(processor.batch_decode(batch['labels'], skip_special_tokens=True))
# Compute WER as MER proxy
wer_score = jiwer.wer(refs, preds)
print(f"Test WER/MER: {wer_score:.2%}")

Epoch 1/5 avg loss: 2.8533
Epoch 2/5 avg loss: 2.8856
Epoch 3/5 avg loss: 2.7823
Epoch 4/5 avg loss: 2.8041
Epoch 5/5 avg loss: 2.6986
Test WER/MER: 721.39%


# Consolidated pipeline (small + LoRA + attention_mask)

In [38]:

# %% [markdown]
# 2. Imports

import os
import json
import numpy as np
import torch
from datasets import load_dataset, Audio
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from peft import LoraConfig, get_peft_model, TaskType
import jiwer

# %% [markdown]
# 3. Data Manifest Generation

# %%
#DATA_ROOT = 'data'
OUT_DIR = 'hf_manifests'

os.makedirs(OUT_DIR, exist_ok=True)
for split in ['train_split', 'test_split']:
    transcripts_dir = os.path.join(split, 'transcripts')
    manifest = os.path.join(OUT_DIR, f'{split}.jsonl')
    with open(manifest, 'w') as fw:
        wav = os.path.join(transcripts_dir, 'wav.scp')
        seg = os.path.join(transcripts_dir, 'segments')
        txt = os.path.join(transcripts_dir, 'text')
        wav_map = dict(line.strip().split(' ',1) for line in open(wav))
        segments = {u:(r,float(s),float(e)) for u,r,s,e in (l.split() for l in open(seg))}
        transcripts = {}
        for line in open(txt):
            parts = line.strip().split(' ',1)
            transcripts[parts[0]] = parts[1] if len(parts)>1 else ''
        for utt,(rec,st,ed) in segments.items():
            entry = {
                'id': utt,
                'audio': {'path': os.path.join(transcripts_dir, wav_map[rec]), 'offset': st, 'duration': ed-st},
                'transcript': transcripts.get(utt,'')
            }
            fw.write(json.dumps(entry)+'\n')
    print(f'Wrote manifest: {manifest}')

# 4. Load Data & Initialize Model
# Load HF dataset, initialize Whisper, freeze base, and apply LoRA. Ensure transcription mode.

# %%
# Dataset
files = {s: os.path.join(OUT_DIR, f'{s}.jsonl') for s in ['train_split','test_split']}
ds = load_dataset('json', data_files=files, split=None)
for s in ds: ds[s] = ds[s].cast_column('audio', Audio(sampling_rate=16000))
# Processor & Model
model_nm = 'openai/whisper-small'
processor = WhisperProcessor.from_pretrained(
    model_nm,
    language='hindi',   # force transcription, not translation
    task='transcribe'
)
model = WhisperForConditionalGeneration.from_pretrained(model_nm)
# Freeze parameters
for p in model.parameters(): p.requires_grad=False
model.config.forced_decoder_ids = processor.get_decoder_prompt_ids(language='hindi', task='transcribe')
# LoRA adapters
lora_cfg = LoraConfig(task_type=TaskType.SEQ_2_SEQ_LM, inference_mode=False, r=16, lora_alpha=32, lora_dropout=0.1, target_modules=['q_proj','v_proj'])
model = get_peft_model(model, lora_cfg)
model.to('cuda' if torch.cuda.is_available() else 'cpu')

#5. Preprocessing & Collation
#Extract features, tokenize, and build attention masks.

# %%
def preprocess(batch):
    arr = batch['audio']['array']
    sr  = batch['audio']['sampling_rate']
    feats = processor.feature_extractor(arr, sampling_rate=sr, return_tensors='pt').input_features[0]
    labs  = processor.tokenizer(batch['transcript'], return_tensors='pt', padding=False).input_ids[0]
    return {'input_features': feats, 'labels': labs}

train_ds = ds['train_split'].map(preprocess, remove_columns=ds['train_split'].column_names)
test_ds  = ds['test_split'].map(preprocess,  remove_columns=ds['test_split'].column_names)

from torch.utils.data import DataLoader

def collate_fn(batch):
    import torch
    feats, masks, labels = [], [], []
    # prepare features and mask
    for b in batch:
        f = b['input_features']
        if not torch.is_tensor(f): f = torch.tensor(f)
        feats.append(f);
    max_t = max(f.shape[0] for f in feats)
    feat_dim = feats[0].shape[1]
    for f in feats:
        pad = torch.zeros((max_t - f.shape[0], feat_dim), dtype=f.dtype)
        feats_p = torch.cat([f, pad], dim=0)
        mask = torch.cat([torch.ones(f.shape[0], dtype=torch.bool), torch.zeros(max_t-f.shape[0], dtype=torch.bool)])
        masks.append(mask)
        feats[padded_feats.index(f)] if False else None
    input_features = torch.stack([torch.cat([f, torch.zeros((max_t-f.shape[0],feat_dim),dtype=f.dtype)],dim=0) for f in feats])
    attention_mask  = torch.stack(masks)
    # prepare labels
    for b in batch:
        l = b['labels']
        if not torch.is_tensor(l): l = torch.tensor(l)
        labels.append(l)
    max_l = max(l.shape[0] for l in labels)
    labels = torch.stack([torch.cat([l, torch.full((max_l-l.shape[0],), -100, dtype=l.dtype)], dim=0) for l in labels])
    return {'input_features': input_features, 'attention_mask': attention_mask, 'labels': labels}

#6. Manual Training & Generation
# Use scheduler, include attention_mask in forward and generate.

from torch.optim import AdamW
from transformers import get_linear_schedule_with_warmup

# DataLoader
device = 'cuda' if torch.cuda.is_available() else 'cpu'
dl_train = DataLoader(train_ds, batch_size=8, shuffle=True, collate_fn=collate_fn)
dl_test  = DataLoader(test_ds, batch_size=4, collate_fn=collate_fn)
# Optimizer & scheduler
opt = AdamW(model.parameters(), lr=5e-5)
steps = len(dl_train)*10  # 10 epochs improving
sched = get_linear_schedule_with_warmup(opt, num_warmup_steps=500, num_training_steps=steps)
# Underlying model bypass
base = model.model if hasattr(model,'model') else model.base_model if hasattr(model,'base_model') else model
base.to(device)

# Training loop
for epoch in range(10):
    base.train(); tot=0.0
    for b in dl_train:
        opt.zero_grad()
        feats = b['input_features'].to(device)
        mask  = b['attention_mask'].to(device)
        labs  = b['labels'].to(device)
        out = base(input_features=feats, attention_mask=mask, labels=labs)
        loss = out.loss; loss.backward(); opt.step(); sched.step(); tot+=loss.item()
    print(f"Epoch {epoch+1} avg loss: {tot/len(dl_train):.4f}")

# Evaluation
tot_preds, tot_refs = [], []
base.eval()
with torch.no_grad():
    for b in dl_test:
        feats = b['input_features'].to(device)
        mask  = b['attention_mask'].to(device)
        gen_ids = base.generate(input_features=feats, attention_mask=mask)
        tot_preds.extend(processor.batch_decode(gen_ids, skip_special_tokens=True))
        tot_refs.extend(processor.batch_decode(b['labels'], skip_special_tokens=True))
# WER/CER
print("Test MER:", jiwer.wer(tot_refs, tot_preds))



Wrote manifest: hf_manifests/train_split.jsonl
Wrote manifest: hf_manifests/test_split.jsonl


Generating train_split split: 0 examples [00:00, ? examples/s]

Generating test_split split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/209 [00:00<?, ? examples/s]

Map:   0%|          | 0/209 [00:00<?, ? examples/s]

Epoch 1 avg loss: 4.6813
Epoch 2 avg loss: 4.5931
Epoch 3 avg loss: 4.5343
Epoch 4 avg loss: 4.4618
Epoch 5 avg loss: 4.1693
Epoch 6 avg loss: 3.7791
Epoch 7 avg loss: 3.4117
Epoch 8 avg loss: 3.1597
Epoch 9 avg loss: 2.9445
Epoch 10 avg loss: 2.8437
Test MER: 5.094958394517866


# I’ve replaced the collate_fn with a corrected implementation that:

Properly converts lists to tensors,

Pads input_features and builds attention_mask,

Pads labels with -100.

In [40]:
from torch.utils.data import DataLoader

def collate_fn(batch):
    import torch
    # Collect input_features and build attention masks
    feats, labels_list = [], []
    for b in batch:
        f = b['input_features']
        if not torch.is_tensor(f):
            f = torch.tensor(f, dtype=torch.float)
        feats.append(f)
        l = b['labels']
        if not torch.is_tensor(l):
            l = torch.tensor(l, dtype=torch.long)
        labels_list.append(l)
    # Pad features
    max_t = max(f.shape[0] for f in feats)
    feat_dim = feats[0].shape[1]
    padded_feats, masks = [], []
    for f in feats:
        pad_len = max_t - f.shape[0]
        pad_feat = torch.zeros((pad_len, feat_dim), dtype=f.dtype)
        padded = torch.cat([f, pad_feat], dim=0)
        padded_feats.append(padded)
        mask = torch.cat([torch.ones(f.shape[0], dtype=torch.bool), torch.zeros(pad_len, dtype=torch.bool)])
        masks.append(mask)
    input_features = torch.stack(padded_feats)
    attention_mask = torch.stack(masks)
    # Pad labels with -100
    max_l = max(l.shape[0] for l in labels_list)
    padded_labels = []
    for l in labels_list:
        pad_len = max_l - l.shape[0]
        pad_label = torch.full((pad_len,), -100, dtype=l.dtype)
        padded_labels.append(torch.cat([l, pad_label], dim=0))
    labels = torch.stack(padded_labels)
    return {'input_features': input_features, 'attention_mask': attention_mask, 'labels': labels}

# %% [markdown]
#6. Manual Training & Generation

# %%
from torch.optim import AdamW
from transformers import get_linear_schedule_with_warmup

# DataLoader
device = 'cuda' if torch.cuda.is_available() else 'cpu'
dl_train = DataLoader(train_ds, batch_size=8, shuffle=True, collate_fn=collate_fn)
dl_test  = DataLoader(test_ds, batch_size=4, collate_fn=collate_fn)
# Optimizer & scheduler
opt = AdamW(model.parameters(), lr=5e-5)
steps = len(dl_train)*10  # 10 epochs improving
sched = get_linear_schedule_with_warmup(opt, num_warmup_steps=500, num_training_steps=steps)
# Underlying model bypass
base = model.model if hasattr(model,'model') else model.base_model if hasattr(model,'base_model') else model
base.to(device)

# Training loop
for epoch in range(10):
    base.train(); tot=0.0
    for b in dl_train:
        opt.zero_grad()
        feats = b['input_features'].to(device)
        mask  = b['attention_mask'].to(device)
        labs  = b['labels'].to(device)
        out = base(input_features=feats, attention_mask=mask, labels=labs)
        loss = out.loss; loss.backward(); opt.step(); sched.step(); tot+=loss.item()
    print(f"Epoch {epoch+1} avg loss: {tot/len(dl_train):.4f}")

# Evaluation
tot_preds, tot_refs = [], []
base.eval()
with torch.no_grad():
    for b in dl_test:
        feats = b['input_features'].to(device)
        mask  = b['attention_mask'].to(device)
        gen_ids = base.generate(input_features=feats, attention_mask=mask)
        tot_preds.extend(processor.batch_decode(gen_ids, skip_special_tokens=True))
        tot_refs.extend(processor.batch_decode(b['labels'], skip_special_tokens=True))
# WER/CER
print("Test MER:", jiwer.wer(tot_refs, tot_preds))

Epoch 1 avg loss: 2.8489
Epoch 2 avg loss: 2.8409
Epoch 3 avg loss: 2.8025
Epoch 4 avg loss: 2.7719
Epoch 5 avg loss: 2.7238
Epoch 6 avg loss: 2.6938
Epoch 7 avg loss: 2.6426
Epoch 8 avg loss: 2.5853
Epoch 9 avg loss: 2.5721
Epoch 10 avg loss: 2.5139
Test MER: 15.167400881057269


# I’ve integrated SpecAugment into the training loop—applying frequency and time masking at each batch—and bumped to 15 epochs with a lower LR of 3e-5. Run this updated code; let’s see the new MER. If we still need improvements, we can explore model scaling or LM rescoring next

In [42]:
# 6. Manual Training & Generation
# Use scheduler, include attention_mask in forward and generate.

# %%
from torch.optim import AdamW
from transformers import get_linear_schedule_with_warmup

# DataLoader
device = 'cuda' if torch.cuda.is_available() else 'cpu'
dl_train = DataLoader(train_ds, batch_size=8, shuffle=True, collate_fn=collate_fn)
dl_test  = DataLoader(test_ds, batch_size=4, collate_fn=collate_fn)
# Optimizer & scheduler
opt = AdamW(model.parameters(), lr=5e-5)
steps = len(dl_train)*10  # 10 epochs improving
sched = get_linear_schedule_with_warmup(opt, num_warmup_steps=500, num_training_steps=steps)
# Underlying model bypass
base = model.model if hasattr(model,'model') else model.base_model if hasattr(model,'base_model') else model
base.to(device)

# %% [markdown]
## 6. Manual Training & Generation (with SpecAugment)
# Use scheduler, include attention_mask in forward and generate, and apply SpecAugment to training audio features.

# %%
from torch.optim import AdamW
from transformers import get_linear_schedule_with_warmup
import torchaudio.transforms as T

# SpecAugment transforms
freq_mask = T.FrequencyMasking(freq_mask_param=27)
time_mask = T.TimeMasking(time_mask_param=100)

# DataLoader
device = 'cuda' if torch.cuda.is_available() else 'cpu'
dl_train = DataLoader(train_ds, batch_size=8, shuffle=True, collate_fn=collate_fn)
dl_test  = DataLoader(test_ds, batch_size=4, collate_fn=collate_fn)
# Optimizer & scheduler
opt = AdamW(model.parameters(), lr=3e-5)
steps = len(dl_train)*15  # 15 epochs
sched = get_linear_schedule_with_warmup(opt, num_warmup_steps=500, num_training_steps=steps)
# Underlying model bypass
base = model.model if hasattr(model,'model') else model.base_model if hasattr(model,'base_model') else model
base.to(device)

# Training loop with SpecAugment
epochs = 15
for epoch in range(epochs:=epochs if 'epochs' in locals() else 15):
    base.train(); tot_loss=0.0
    for b in dl_train:
        opt.zero_grad()
        feats = b['input_features'].to(device)           # (B, T, D)
        # Apply SpecAugment: need (B, D, T)
        feats_aug = feats.permute(0,2,1)
        feats_aug = freq_mask(feats_aug)
        feats_aug = time_mask(feats_aug)
        feats_aug = feats_aug.permute(0,2,1)
        mask  = b['attention_mask'].to(device)
        labs  = b['labels'].to(device)
        out = base(input_features=feats_aug, attention_mask=mask, labels=labs)
        loss = out.loss; loss.backward(); opt.step(); sched.step(); tot_loss+=loss.item()
    print(f"Epoch {epoch+1}/{epochs} avg loss: {tot_loss/len(dl_train):.4f}")

# %% [markdown]
## 7. Evaluation

# Evaluate without augmentation
dl_test = DataLoader(test_ds, batch_size=4, collate_fn=collate_fn)
base.eval(); preds, refs = [], []
with torch.no_grad():
    for b in dl_test:
        feats = b['input_features'].to(device)
        mask  = b['attention_mask'].to(device)
        gen_ids = base.generate(input_features=feats, attention_mask=mask)
        preds.extend(processor.batch_decode(gen_ids, skip_special_tokens=True))
        refs.extend(processor.batch_decode(b['labels'], skip_special_tokens=True))
# Compute WER/CER
print("Test MER:", jiwer.wer(refs, preds))

tot_preds, tot_refs = [], []
base.eval()
with torch.no_grad():
    for b in dl_test:
        feats = b['input_features'].to(device)
        mask  = b['attention_mask'].to(device)
        gen_ids = base.generate(input_features=feats, attention_mask=mask)
        tot_preds.extend(processor.batch_decode(gen_ids, skip_special_tokens=True))
        tot_refs.extend(processor.batch_decode(b['labels'], skip_special_tokens=True))
# WER/CER
print("Test MER:", jiwer.wer(tot_refs, tot_preds))

Epoch 1/15 avg loss: 3.1173
Epoch 2/15 avg loss: 3.0652
Epoch 3/15 avg loss: 3.0307
Epoch 4/15 avg loss: 2.9093
Epoch 5/15 avg loss: 3.0522
Epoch 6/15 avg loss: 2.8615
Epoch 7/15 avg loss: 2.9415
Epoch 8/15 avg loss: 2.9710
Epoch 9/15 avg loss: 2.7921
Epoch 10/15 avg loss: 2.7843
Epoch 11/15 avg loss: 2.7789
Epoch 12/15 avg loss: 2.6767
Epoch 13/15 avg loss: 2.6572
Epoch 14/15 avg loss: 2.6256
Epoch 15/15 avg loss: 2.5515
Test MER: 1.066079295154185
Test MER: 1.066079295154185


# I’ve upgraded the base model to openai/whisper-medium for stronger acoustic features—simply swap out “small” for “medium” when loading the processor and model. 

In [45]:
# 4. Load Data & Initialize Model

# %%
# Dataset
files = {s: os.path.join(OUT_DIR, f'{s}.jsonl') for s in ['train_split','test_split']}
ds = load_dataset('json', data_files=files, split=None)
for s in ds: ds[s] = ds[s].cast_column('audio', Audio(sampling_rate=16000))
# Processor & Model
model_nm = 'openai/whisper-medium'  # Upgrade to medium for richer representations
processor = WhisperProcessor.from_pretrained(
    model_nm,
    language='hindi',   # force transcription, not translation
    task='transcribe'
)
model = WhisperForConditionalGeneration.from_pretrained(model_nm)
# Freeze parameters
for p in model.parameters(): p.requires_grad=False
model.config.forced_decoder_ids = processor.get_decoder_prompt_ids(language='hindi', task='transcribe')
# LoRA adapters
lora_cfg = LoraConfig(task_type=TaskType.SEQ_2_SEQ_LM, inference_mode=False, r=16, lora_alpha=32, lora_dropout=0.1, target_modules=['q_proj','v_proj'])
model = get_peft_model(model, lora_cfg)
model.to('cuda' if torch.cuda.is_available() else 'cpu')

# %% [markdown]
# 5. Preprocessing & Collation
# Extract features, tokenize, and build attention masks.

# %%
def preprocess(batch):
    arr = batch['audio']['array']
    sr  = batch['audio']['sampling_rate']
    feats = processor.feature_extractor(arr, sampling_rate=sr, return_tensors='pt').input_features[0]
    labs  = processor.tokenizer(batch['transcript'], return_tensors='pt', padding=False).input_ids[0]
    return {'input_features': feats, 'labels': labs}

train_ds = ds['train_split'].map(preprocess, remove_columns=ds['train_split'].column_names)
test_ds  = ds['test_split'].map(preprocess,  remove_columns=ds['test_split'].column_names)

from torch.utils.data import DataLoader

def collate_fn(batch):
    import torch
    # Collect input_features and build attention masks
    feats, labels_list = [], []
    for b in batch:
        f = b['input_features']
        if not torch.is_tensor(f):
            f = torch.tensor(f, dtype=torch.float)
        feats.append(f)
        l = b['labels']
        if not torch.is_tensor(l):
            l = torch.tensor(l, dtype=torch.long)
        labels_list.append(l)
    # Pad features
    max_t = max(f.shape[0] for f in feats)
    feat_dim = feats[0].shape[1]
    padded_feats, masks = [], []
    for f in feats:
        pad_len = max_t - f.shape[0]
        pad_feat = torch.zeros((pad_len, feat_dim), dtype=f.dtype)
        padded = torch.cat([f, pad_feat], dim=0)
        padded_feats.append(padded)
        mask = torch.cat([torch.ones(f.shape[0], dtype=torch.bool), torch.zeros(pad_len, dtype=torch.bool)])
        masks.append(mask)
    input_features = torch.stack(padded_feats)
    attention_mask = torch.stack(masks)
    # Pad labels with -100
    max_l = max(l.shape[0] for l in labels_list)
    padded_labels = []
    for l in labels_list:
        pad_len = max_l - l.shape[0]
        pad_label = torch.full((pad_len,), -100, dtype=l.dtype)
        padded_labels.append(torch.cat([l, pad_label], dim=0))
    labels = torch.stack(padded_labels)
    return {'input_features': input_features, 'attention_mask': attention_mask, 'labels': labels}

# %% [markdown]
# 6. Manual Training & Generation
# Use scheduler, include attention_mask in forward and generate.

# %%
from torch.optim import AdamW
from transformers import get_linear_schedule_with_warmup

# DataLoader
device = 'cuda' if torch.cuda.is_available() else 'cpu'
dl_train = DataLoader(train_ds, batch_size=8, shuffle=True, collate_fn=collate_fn)
dl_test  = DataLoader(test_ds, batch_size=4, collate_fn=collate_fn)
# Optimizer & scheduler
opt = AdamW(model.parameters(), lr=5e-5)
steps = len(dl_train)*10  # 10 epochs improving
sched = get_linear_schedule_with_warmup(opt, num_warmup_steps=500, num_training_steps=steps)
# Underlying model bypass
base = model.model if hasattr(model,'model') else model.base_model if hasattr(model,'base_model') else model
base.to(device)

# %% [markdown]
## 6. Manual Training & Generation (with SpecAugment)
# Use scheduler, include attention_mask in forward and generate, and apply SpecAugment to training audio features.

# %%
from torch.optim import AdamW
from transformers import get_linear_schedule_with_warmup
import torchaudio.transforms as T

# SpecAugment transforms
freq_mask = T.FrequencyMasking(freq_mask_param=27)
time_mask = T.TimeMasking(time_mask_param=100)

# DataLoader
device = 'cuda' if torch.cuda.is_available() else 'cpu'
dl_train = DataLoader(train_ds, batch_size=8, shuffle=True, collate_fn=collate_fn)
dl_test  = DataLoader(test_ds, batch_size=4, collate_fn=collate_fn)
# Optimizer & scheduler
opt = AdamW(model.parameters(), lr=3e-5)
steps = len(dl_train)*15  # 15 epochs
sched = get_linear_schedule_with_warmup(opt, num_warmup_steps=500, num_training_steps=steps)
# Underlying model bypass
base = model.model if hasattr(model,'model') else model.base_model if hasattr(model,'base_model') else model
base.to(device)

# Training loop with SpecAugment
epochs = 15
for epoch in range(epochs:=epochs if 'epochs' in locals() else 15):
    base.train(); tot_loss=0.0
    for b in dl_train:
        opt.zero_grad()
        feats = b['input_features'].to(device)           # (B, T, D)
        # Apply SpecAugment: need (B, D, T)
        feats_aug = feats.permute(0,2,1)
        feats_aug = freq_mask(feats_aug)
        feats_aug = time_mask(feats_aug)
        feats_aug = feats_aug.permute(0,2,1)
        mask  = b['attention_mask'].to(device)
        labs  = b['labels'].to(device)
        out = base(input_features=feats_aug, attention_mask=mask, labels=labs)
        loss = out.loss; loss.backward(); opt.step(); sched.step(); tot_loss+=loss.item()
    print(f"Epoch {epoch+1}/{epochs} avg loss: {tot_loss/len(dl_train):.4f}")

# %% [markdown]
## 7. Evaluation

# Evaluate without augmentation
dl_test = DataLoader(test_ds, batch_size=4, collate_fn=collate_fn)
base.eval(); preds, refs = [], []
with torch.no_grad():
    for b in dl_test:
        feats = b['input_features'].to(device)
        mask  = b['attention_mask'].to(device)
        gen_ids = base.generate(input_features=feats, attention_mask=mask)
        preds.extend(processor.batch_decode(gen_ids, skip_special_tokens=True))
        refs.extend(processor.batch_decode(b['labels'], skip_special_tokens=True))
# Compute WER/CER
print("Test MER:", jiwer.wer(refs, preds))

tot_preds, tot_refs = [], []
base.eval()
with torch.no_grad():
    for b in dl_test:
        feats = b['input_features'].to(device)
        mask  = b['attention_mask'].to(device)
        gen_ids = base.generate(input_features=feats, attention_mask=mask)
        tot_preds.extend(processor.batch_decode(gen_ids, skip_special_tokens=True))
        tot_refs.extend(processor.batch_decode(b['labels'], skip_special_tokens=True))
# WER/CER
print("Test MER:", jiwer.wer(tot_refs, tot_preds))

Epoch 1/15 avg loss: 4.6990
Epoch 2/15 avg loss: 4.5239
Epoch 3/15 avg loss: 4.6079
Epoch 4/15 avg loss: 4.4906
Epoch 5/15 avg loss: 4.3763
Epoch 6/15 avg loss: 4.2706
Epoch 7/15 avg loss: 4.1671
Epoch 8/15 avg loss: 3.7595
Epoch 9/15 avg loss: 3.5597
Epoch 10/15 avg loss: 3.5794
Epoch 11/15 avg loss: 3.4650
Epoch 12/15 avg loss: 3.3509
Epoch 13/15 avg loss: 3.2239
Epoch 14/15 avg loss: 3.1868
Epoch 15/15 avg loss: 3.0129
Test MER: 3.3416544297601565
Test MER: 3.3416544297601565
